# Backtest

In [ ]:
from system.agent import Agent
from system.env import Env, GEnv

from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


START = "2022-01-01"
END = "2023-10-01"
TICKERS = ["BTC", "ETH", "SOL", "DASH", "MATIC", "TWT", "GALA", "EGLD", "XMR"]

INTERVAL = "1d"
CAPITAL = 100

config = {
    "symbols" : TICKERS, "interval" : INTERVAL, "start" : START, "end" : END
    }
env = GEnv(**config)


sub_env_1 = Env(symbol="BTC", interval=INTERVAL, start=START, end=END)
sub_env_2 = Env(symbol="ETH", interval=INTERVAL, start=START, end=END)
sub_env_3 = Env(symbol="SOL", interval=INTERVAL, start=START, end=END)
sub_env_4 = Env(symbol="EGLD", interval=INTERVAL, start=START, end=END)


# Agent details
CAPITAL = 100
AGENT_PARAMS = [
    [(1, "BTC"), CAPITAL, sub_env_1],
    [(2, "ETH"), CAPITAL,sub_env_2],
    [(3, "SOL"), CAPITAL,sub_env_3],
    [(4, "EGLD"), CAPITAL,sub_env_4]
]


STRATEGIES = [
    ("MOMENTUM", 3, {"floor" : 4}),
    ("TRIPLEMA", (3, 7, 14), {"floor" : 4}),
    ("MOMENTUM", 8, {"floor" : 4}),
    ("TRIPLEMA", (14, 21, 35), {"floor" : 4})
]

AGENT = []
for agent_x, strategie in zip(AGENT_PARAMS, STRATEGIES):
    clear_output()
    print(agent_x)
    agent = Agent(*agent_x)
    agent.update_policy(*strategie)
    agent.run_episode()
    AGENT.append(agent)
    
    

In [ ]:
AGENT[0].view_report()

In [ ]:
#AGENT[1].env.post_event.tradeData[AGENT[1].env.post_event.tradeData["session"]==1]
AGENT[1].env.post_event.tradeData.iloc[:53]

In [ ]:
AGENT[1].view_report()

In [ ]:
AGENT[3].view_report()

In [ ]:
#AGENT[3].post_event.tradeData[AGENT[3].post_event.tradeData["session"] == 1].iloc[2:12]

In [ ]:
env.globalReport()

# Agent : Learning

In [11]:
from system.agent import Agent
from system.env import Env, GEnv

from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


START = "2024"
END = "2023"
TICKERS = ["BTC", "ETH", "SOL", "QNT", "DASH", "MATIC", "TWT", "GALA", "EGLD", "XMR"]

SYMBOL = "QNT"
INTERVAL = "1d"
CAPITAL = 1000


config = {
    "symbols" : TICKERS, "interval" : INTERVAL, "start" : START, "end" : END
    }
env = GEnv(**config)

sub_env = Env(symbol=SYMBOL, interval=INTERVAL, start=START, end=END)

# Agent details
CAPITAL = 1000
LEVERAGE = 5

n_SESSION = 100

AGENT_PARAMS = [
    [(1, SYMBOL), CAPITAL, sub_env, n_SESSION]
]




STRATEGIES = [
    ("MOMENTUM", 3, {"floor" : 4}),
    ("TRIPLEMA", (3, 7, 14), {"floor" : 4}),
    ("MOMENTUM", 8, {"floor" : 4}),
    ("TRIPLEMA", (14, 21, 35), {"floor" : 4})
]

STRATEGIES = [
    ("TRIPLEMA", (3, 7, 14), {"floor" : 4})
]

STRATEGIES = [
    ("MOMENTUM", 3, {"floor" : 4})
]

AGENT = []
for agent_x, strategie in zip(AGENT_PARAMS, STRATEGIES):
    print(agent_x)
    agent = Agent(*agent_x)
    agent.update_policy(*strategie)
    agent.run_episode()
    AGENT.append(agent)
    
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[(1, 'QNT'), 1000, <system.env.Env object at 0x00000138162DE650>, 100]
except :  
N : 0


In [ ]:
AGENT[0].view_report()

In [2]:
session = AGENT[0].post_event.session
session

""


In [3]:
(session["nbTrades"].sum() / dd.shape[0]) * 100

KeyError: 'nbTrades'

In [4]:
dd = AGENT[0].env.post_event.tradeData
dd.head(15)

""


In [ ]:
port = AGENT[0].env.post_event.portfolioData
port

#### --------------------------------------

# *Multi-Agent* v2(1)

In [ ]:
from magent import MAgentThread
from master import  MasterAgentThread
from system.env import Env, SubEnv

from threading import Thread, Condition, Event, Barrier, Semaphore

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


TICKERS = ["BTC", "ETH", "SOL", "DASH"]
START = "2022-01-01"
END = "2023-10-01"
INTERVAL = "1d"
CAPITAL = 100

config = {"capital" : CAPITAL, "interval" : INTERVAL,
          "symbols" : TICKERS, "start" : START, "end" : END}
env = Env(**config)

sub_env_1 = SubEnv(symbol="BTC", capital=CAPITAL, interval=INTERVAL, start=START, end=END)
sub_env_2 = SubEnv(symbol="ETH", capital=CAPITAL, interval=INTERVAL, start=START, end=END)
sub_env_3 = SubEnv(symbol="SOL", capital=CAPITAL, interval=INTERVAL, start=START, end=END)
sub_env_4 = SubEnv(symbol="EGLD", capital=CAPITAL, interval=INTERVAL, start=START, end=END)

# Agent details
AGENT_PARAMS = [
    [(1, "BTC"), sub_env_1],
    [(2, "ETH"), sub_env_2],
    [(3, "SOL"), sub_env_3],
    [(4, "EGLD"), sub_env_4]
]

N = len(AGENT_PARAMS)

STRATEGIES = [
    ((1, "BTC"), "MOMENTUM", 3),
    ((2, "ETH"), "TRIPLEMA", (3, 7, 14)),
    ((3, "SOL"), "MOMENTUM", 8),
    ((4, "EGLD"), "TRIPLEMA", (14, 21, 35))
]

In [ ]:
master = MasterAgentThread(env)
master.set_barrier(n=N)

for agent_param, strategie in zip(AGENT_PARAMS, STRATEGIES):
    master.add_agent(*agent_param)
    master.update_params(*strategie)


master.active_agents()
master.run()

In [ ]:
master.agents

In [ ]:
#master.agents[(1, "BTC")].env.get_viz(1, "BTC")
master.agents[(AGENT_PARAMS[0][0])].env.get_viz(*AGENT_PARAMS[0][0])

In [ ]:
master.agents[(AGENT_PARAMS[1][0])].env.get_viz(*AGENT_PARAMS[1][0])

In [ ]:
master.agents[(AGENT_PARAMS[2][0])].env.get_viz(*AGENT_PARAMS[2][0])

In [ ]:
master.agents[(AGENT_PARAMS[3][0])].env.get_viz(*AGENT_PARAMS[3][0])

In [ ]:
master.agents[(4, "EGLD")].env.journal.portfolioData
#master.agents[(3, "SOL")].env.journal.tradesData

In [ ]:
master.agents

In [ ]:
master.agents[(3, "SOL")].env.journal.portfolioData

# *End*

In [ ]:
import threading

threading.enumerate()

# *Multi-Agent v3* Multiprocessing

In [ ]:
import multiprocessing
from multiprocessing import Process, Manager, Queue, Barrier, Event, Semaphore



# Plot

In [ ]:
import plotly.graph_objects as go

categories = ['processing cost','mechanical properties','chemical stability',
              'thermal stability', 'device integration']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=[1, 5, 2, 2, 3],
      theta=categories,
      fill='toself',
      name='Product A'
))
fig.add_trace(go.Scatterpolar(
      r=[4, 3, 2.5, 1, 2],
      theta=categories,
      fill='toself',
      name='Product B'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 5]
    )),
  showlegend=False
)

fig.show()

In [ ]:
class MyGenerator:
    def __init__(self, n):
        self.n = n
        self.current = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.current < self.n:
            result = self.current
            self.current += 1
            return result
        else:
            raise StopIteration

# Créez une instance de la classe MyGenerator
dix_nombres = MyGenerator(10)

# Utilisez une boucle for pour obtenir les valeurs
for nombre in dix_nombres:
    print(nombre)


In [ ]:
import multiprocessing

multiprocessing.cpu_count()

In [ ]:
import os 

os.cpu_count()

In [ ]:
import threading

class Master:
    def __init__(self, condition, event):
        self.condition = condition
        self.event = event

    def run(self):
        for _ in range(10):
            with self.condition:
                print("Maître : J'attends que les agents soient prêts.")
                self.condition.wait()
                print("Maître : Autorisation d'exécuter le travail.")
                self.event.set()
                self.condition.notify_all()

class Agent:
    def __init__(self, agent_id, condition, event):
        self.agent_id = agent_id
        self.condition = condition
        self.event = event

    def run(self):
        for _ in range(10):
            with self.condition:
                print(f"Agent {self.agent_id} : Prêt à travailler.")
                self.condition.notify()
                self.condition.wait()
                self.event.wait()
                self.event.clear()
                print(f"Agent {self.agent_id} : Travail en cours.")
                self.condition.notify()
                self.condition.wait()
                print(f"Agent {self.agent_id} : Travail terminé.")

# Créez une Condition pour coordonner les agents et le maître
condition = threading.Condition()

# Créez un Event pour autoriser les agents à travailler
event = threading.Event()

# Créez le maître et les agents
master = Master(condition, event)
agents = [Agent(i, condition, event) for i in range(1, 4)]

# Créez des threads pour le maître et les agents
master_thread = threading.Thread(target=master.run)
agent_threads = [threading.Thread(target=agent.run) for agent in agents]

# Démarrez les threads
master_thread.start()
for thread in agent_threads:
    thread.start()

# Attendez que tous les threads se terminent
master_thread.join()
for thread in agent_threads:
    thread.join()

print("Tous les agents et le maître ont terminé.")


In [ ]:
import time
from threading import Thread
from queue import Queue

class Agent:
    def __init__(self, master_queue, name):
        self.master_queue = master_queue
        self.name = name

    def do_task(self, task):
        print(f"Agent {self.name} received task: {task}")
        # Simulating task execution
        time.sleep(2)
        print(f"Agent {self.name} completed task: {task}")
        # Inform the master that the task is completed
        self.master_queue.put((self.name, task))


class Master:
    def __init__(self):
        self.agent_queues = {}

    def add_agent(self, agent):
        agent_queue = Queue()
        self.agent_queues[agent.name] = agent_queue
        Thread(target=self.listen_to_agent, args=(agent, agent_queue)).start()

    def assign_task(self, agent_name, task):
        agent_queue = self.agent_queues.get(agent_name)
        if agent_queue:
            print(f"Master assigned task: {task} to Agent {agent_name}")
            agent_queue.put(task)
        else:
            print(f"Agent {agent_name} not found.")

    def listen_to_agent(self, agent, agent_queue):
        while True:
            task = agent_queue.get()
            if task is None:
                break
            print(f"Master received task completion report from Agent {agent.name} for task: {task}")


# Exemple d'utilisation
if __name__ == "__main__":
    master = Master()

    agents = [Agent(master_queue=master, name=f"Agent{i+1}") for i in range(4)]

    for agent in agents:
        master.add_agent(agent)

    master.assign_task("Agent1", "Task1")
    master.assign_task("Agent2", "Task2")
    master.assign_task("Agent3", "Task3")
    master.assign_task("Agent4", "Task4")

    # Attendez la fin des tâches avant de quitter
    time.sleep(3)  # Simuler le temps nécessaire pour terminer les tâches
    for agent in agents:
        master.add_agent(agent)  # Envoyer un signal de fin à tous les agents


In [ ]:
import multiprocessing
import time
import random

class Agent(multiprocessing.Process):
    def __init__(self, agent_id, master_queue, agent_queue):
        super().__init__()
        self.agent_id = agent_id
        self.master_queue = master_queue
        self.agent_queue = agent_queue

    def run(self):
        while True:
            instruction = self.master_queue.get()

            if instruction == "QUIT":
                break

            # Ajoutez des conditions en fonction de l'instruction
            if instruction == "Commande 1":
                result = f"Agent {self.agent_id} exécute la Commande 1 spécifique."
            elif instruction == "Commande 2":
                result = f"Agent {self.agent_id} exécute la Commande 2 spécifique."
            else:
                result = f"Agent {self.agent_id} exécute une instruction non gérée."

            self.agent_queue.put(result)

class Master:
    def __init__(self):
        self.master_queue = multiprocessing.Queue()
        self.agent_queues = [multiprocessing.Queue() for _ in range(4)]
        self.agents = [Agent(i, self.master_queue, self.agent_queues[i]) for i in range(4)]

    def start_agents(self):
        for agent in self.agents:
            agent.start()

    def send_instructions(self, instructions):
        for instruction in instructions:
            for agent_queue in self.agent_queues:
                self.master_queue.put(instruction)

            time.sleep(1)  # Attendez un certain temps pour simuler le traitement parallèle

            for _ in range(4):
                report = self.master_queue.get()
                print(report)

    def stop_agents(self):
        for _ in range(4):
            self.master_queue.put("QUIT")

        for agent in self.agents:
            agent.join()

if __name__ == "__main__":
    master_instance = Master()
    master_instance.start_agents()

    instructions = ["Commande 1", "Commande 2", "Commande 3", "Commande 4"]
    master_instance.send_instructions(instructions)

    master_instance.stop_agents()


In [ ]:
import multiprocessing
import time

class Agent(multiprocessing.Process):
    def __init__(self, agent_id, master_event, agent_queue):
        super().__init__()
        self.agent_id = agent_id
        self.master_event = master_event
        self.agent_queue = agent_queue

    def run(self):
        while True:
            # Wait for the event to be set by the master
            self.master_event.wait()

            instruction = self.agent_queue.get()

            if instruction == "QUIT":
                break

            # Your conditions or logic here
            result = f"Agent {self.agent_id} a exécuté l'instruction: {instruction}"

            # Report the result to the master
            self.agent_queue.put(result)

class Master:
    def __init__(self):
        self.master_event = multiprocessing.Event()
        self.agent_queues = [multiprocessing.Queue() for _ in range(4)]
        self.agents = [Agent(i, self.master_event, self.agent_queues[i]) for i in range(4)]

    def start_agents(self):
        for agent in self.agents:
            agent.start()

    def send_instructions(self, instructions):
        for instruction in instructions:
            # Send the instruction to all agents
            for agent_queue in self.agent_queues:
                agent_queue.put(instruction)

            # Set the event to signal agents to start processing
            self.master_event.set()

            time.sleep(1)  # Simulate parallel processing time

            # Clear the event for the next round
            self.master_event.clear()

            # Collect reports from agents
            for _ in range(4):
                report = self.agent_queues[0].get()  # Assuming all agents provide the same result
                print(report)

    def stop_agents(self):
        # Send the QUIT instruction to all agents
        for agent_queue in self.agent_queues:
            agent_queue.put("QUIT")

        # Wait for agents to finish
        for agent in self.agents:
            agent.join()

if __name__ == "__main__":
    master_instance = Master()
    master_instance.start_agents()

    instructions = ["Commande 1", "Commande 2", "Commande 3", "Commande 4"]
    master_instance.send_instructions(instructions)

    master_instance.stop_agents()


In [ ]:


class ProfitManagement:
    def __init__(self, profit_target_percentage, trailing_stop_percentage):
        self.profit_target_percentage = profit_target_percentage
        self.trailing_stop_percentage = trailing_stop_percentage
        self.entry_price = None
        self.profit_target_price = None
        self.trailing_stop_price = None

    def set_entry_price(self, entry_price):
        self.entry_price = entry_price
        self.profit_target_price = entry_price * (1 + self.profit_target_percentage)
        self.trailing_stop_price = entry_price * (1 - self.trailing_stop_percentage)

    def take_profit(self, current_price):
        if current_price >= self.profit_target_price:
            return True
        return False

    def trailing_stop_loss(self, current_price):
        if current_price >= self.trailing_stop_price:
            self.trailing_stop_price = current_price * (1 - self.trailing_stop_percentage)
        return self.trailing_stop_price

# Exemple d'utilisation
profit_target_percentage = 0.05  # Objectif de profit de 5%
trailing_stop_percentage = 0.02  # Trailing stop-loss de 2%

profit_manager = ProfitManagement(profit_target_percentage, trailing_stop_percentage)
profit_manager.set_entry_price(100)  # Prix d'entrée de la position

# Simulation du mouvement des prix
price_history = [100, 110, 120, 115, 125, 130, 135, 130, 140, 145]

for price in price_history:
    if profit_manager.take_profit(price):
        print(f"Prendre profit au prix {price}")
        break
    else:
        trailing_stop_price = profit_manager.trailing_stop_loss(price)
        print(f"Prix actuel : {price}, Trailing stop-loss : {trailing_stop_price}")
